In [3]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install jupyterlab pillow requests


Looking in indexes: https://download.pytorch.org/whl/cu118
     ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
     - -------------------------------------- 0.3/6.2 MB ? eta -:--:--
     ----- ---------------------------------- 0.8/6.2 MB 2.1 MB/s eta 0:00:03
     ----- ---------------------------------- 0.8/6.2 MB 2.1 MB/s eta 0:00:03
     ------ --------------------------------- 1.0/6.2 MB 1.6 MB/s eta 0:00:04
     ---------- ----------------------------- 1.6/6.2 MB 1.5 MB/s eta 0:00:04
     ----------- ---------------------------- 1.8/6.2 MB 1.5 MB/s eta 0:00:03
     ----------- ---------------------------- 1.8/6.2 MB 1.5 MB/s eta 0:00:03
     ------------- -------------------------- 2.1/6.2 MB 1.4 MB/s eta 0:00:03
     --------------- ------------------------ 2.4/6.2 MB 1.3 MB/s eta 0:00:03
     ---------------- ----------------------- 2.6/6.2 MB 1.3 MB/s eta 0:00:03
     -------------------- ------------------- 3.1/6.2 MB 1.4 MB/s eta 0:00:03
     --------------

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\Serhat\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\Serhat\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\Serhat\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\Serhat\anaconda3\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Serhat\anaconda3\Lib\http\client.py", line 479, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "C:\Users\Serhat\anaconda3\Lib\socket.py", line 720, in readinto
    return self._sock.recv_into(b)
      


   ---------------------- ----------------- 1.5/2.7 GB 658.5 kB/s eta 0:30:02
   ---------------------- ----------------- 1.5/2.7 GB 658.5 kB/s eta 0:30:02
   ---------------------- ----------------- 1.5/2.7 GB 659.1 kB/s eta 0:30:00
   ---------------------- ----------------- 1.5/2.7 GB 662.6 kB/s eta 0:29:50
   ---------------------- ----------------- 1.5/2.7 GB 665.8 kB/s eta 0:29:41
   ---------------------- ----------------- 1.5/2.7 GB 664.1 kB/s eta 0:29:45
   ---------------------- ----------------- 1.5/2.7 GB 664.1 kB/s eta 0:29:45
   ---------------------- ----------------- 1.5/2.7 GB 653.4 kB/s eta 0:30:14
   ---------------------- ----------------- 1.5/2.7 GB 653.4 kB/s eta 0:30:14
   ---------------------- ----------------- 1.5/2.7 GB 654.8 kB/s eta 0:30:10
   ---------------------- ----------------- 1.5/2.7 GB 654.8 kB/s eta 0:30:10
   ---------------------- ----------------- 1.5/2.7 GB 654.8 kB/s eta 0:30:10
   ---------------------- ----------------- 1.5/2.7 GB 650.7 kB

In [ ]:
!pip install torch transformers datasets torchvision

In [106]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model ="distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForSequenceClassification.from_pretrained(model).to("cuda" if torch.cuda.is_available() else "cpu")

In [108]:
text= "I feel not good"
inputs = tokenizer(text,return_tensors="pt").to(model.device) #text to numeric value

In [110]:
with torch.no_grad(): #The model operates in inference mode and does not perform training or gradient computation
    outputs = model(**inputs) 
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=1).item()



In [112]:
etiketler = ["Negative", "Positive"]
print("Tahmin:", etiketler[prediction])

Tahmin: Negative


In [126]:
import time 
start = time.time()
with torch.no_grad():
    output = model(**inputs)
end = time.time()

print(end-start)

0.016952991485595703


In [128]:
#original model size
import os
torch.save(model.state_dict(),"origin_model.pt")
print(os.path.getsize("origin_model.pt")/1e6,"MB")

267.860606 MB


In [118]:
from torch.quantization import quantize_dynamic

quantized_model = quantize_dynamic(
    model,
    {torch.nn.Linear},
    dtype = torch.qint8
)

In [120]:
import time 
start = time.time()
with torch.no_grad():
    output = quantized_model(**inputs)
end = time.time()

print(end-start)

0.010969400405883789


In [122]:
#quantized model size
import os
torch.save(quantized_model.state_dict(),"quantized_model.pt")
print(os.path.getsize("quantized_model.pt")/1e6,"MB")

138.715658 MB
